### Compute the mean and std of images in the datasets

In [1]:
import os
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchmetrics
from torch import nn, optim
import pytorch_lightning as pl
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import random_split
from pytorch_lightning.callbacks import ModelCheckpoint
from pathlib import Path as p
import re
from PIL import Image
from torchsummary import summary
from ../Preprocessing_zip import Processor

NUM_WORKERS = os.cpu_count() // 2

# data_path = './Hand extraction/source_box'
# transform_img = transforms.Compose(
#     [transforms.CenterCrop(720),
#      transforms.ToTensor()]
# )
#
# image_data = ImageFolder(
#     root=data_path, transform=transform_img
# )
# # data_processor = Processor(data_path, transforms=transform_img, data_sample=0.5)
# # data_processor.get_data()
# # train_image_data = data_processor.train
#
# image_data_loader = DataLoader(
#     image_data,
#     batch_size=len(image_data),
#     shuffle=False,
#     num_workers=NUM_WORKERS)
#
#
# def mean_std(loader):
#     images, _ = next(iter(loader))
#     # shape of images = [b,c,w,h]
#     mean, std = images.mean([0, 2, 3]), images.std([0, 2, 3])
#     return mean, std
#
#
# IMAGES_MEAN, IMAGES_STD = mean_std(image_data_loader)

BATCH_SIZE = int(2 ** 6)
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
pl.seed_everything(432)
IMAGES_MEAN, IMAGES_STD = torch.tensor([0.0784, 0.0672, 0.0628]), torch.tensor([0.2256, 0.1979, 0.1872])

Global seed set to 432


### Create Lightning Data Module (subclass)

In [2]:
class ImageDataset(pl.LightningDataModule):
    def __init__(self, data_dir: str = None, batch_size: int = BATCH_SIZE, num_workers=NUM_WORKERS):
        super().__init__()
        self.data_dir = data_dir or os.getcwd()
        self.num_workers = num_workers
        self.batch_size = batch_size
        self.train_transform = transforms.Compose(
            [transforms.CenterCrop(200),
             transforms.RandomHorizontalFlip(p=0.1),
             transforms.RandomVerticalFlip(p=0.1),
             transforms.ToTensor(),
             transforms.Normalize(IMAGES_MEAN, IMAGES_STD)]
        )
        self.test_transform = transforms.Compose(
            [transforms.Grayscale(num_output_channels=1),
             transforms.ToTensor(),
             transforms.Normalize(IMAGES_MEAN, IMAGES_STD)]
        )

    def prepare_data(self):
        self.data = ImageFolder(
            self.data_dir, transform=self.train_transform)

    def setup(self, train_ratio: float = 0.9, stage=None):
        if stage == 'fit' or stage is None:
            train_amount = int(len(self.data) * train_ratio)
            self.train, self.test = random_split(
                self.data, [train_amount, len(self.data) - train_amount])
            train_amount = int(len(self.train) * train_ratio)
            self.train, self.val = random_split(
                self.train, [train_amount, len(self.train) - train_amount])
        if stage == 'test' or stage is None:
            pass

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, num_workers=self.num_workers, pin_memory=True,
                          shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size, num_workers=self.num_workers, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size, num_workers=self.num_workers, pin_memory=True)

    def smallest_size(self):
        '''Return the smallest resolution of images in the folder_path'''
        min_size = 10 ** 9
        for file in p(self.data_dir).rglob('*'):
            if file.is_file():
                im = Image.open(file)
                min_size = min(min_size, *im.size)
        return min_size

### Create Lightning Module subclass (model)

In [3]:
'''
We inherit a Resnet model
https://github.com/Stevellen/ResNet-Lightning/blob/master/resnet_classifier.py
https://jarvislabs.ai/blogs/resnet/
https://discuss.pytorch.org/t/transfer-learning-usage-with-different-input-size/20744/3
'''


class Resnet(pl.LightningModule):
    def __init__(self, num_classes=10, model_choice=18, weights=None):
        super().__init__()
        self.__dict__.update(locals())
        model_choices = {
            18: models.resnet18, 34: models.resnet34,
            50: models.resnet50, 101: models.resnet101,
            152: models.resnet152,
            'vgg16': models.vgg16
        }
        self.acc = torchmetrics.Accuracy()
        self.lr = 1e-3
        # instantiate loss criterion
        self.loss = nn.BCEWithLogitsLoss() if num_classes == 2 else nn.CrossEntropyLoss()
        # Do not use a pretrained ResNet backbone
        self.model = model_choices[model_choice](weights=weights)

    def forward(self, X):
        return self.model(X)

    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=self.lr)

    def training_step(self, batch, batch_idx):
        x, y = batch
        preds = self.forward(x)
        if self.num_classes == 2:
            y = F.one_hot(y, num_classes=2).float()
        loss = self.loss(preds, y)
        acc = self.acc(preds, y)
        # Logging the loss
        self.log("train_loss", loss, on_epoch=True, on_step=True, logger=True)
        self.log('train_acc', acc, on_epoch=True, on_step=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        preds = self.forward(x)
        if self.num_classes == 2:
            y = F.one_hot(y, num_classes=2).float()
        loss = self.loss(preds, y)
        acc = self.acc(preds, y)
        # Logging the loss
        self.log("val_loss", loss, on_epoch=True, on_step=True, logger=True)
        self.log('val_acc', acc, on_epoch=True, on_step=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        if self.num_classes == 2:
            y = F.one_hot(y, num_classes=2).float()
        loss = self.loss(preds, y)
        acc = self.acc(preds, y)
        # perform logging
        self.log("test_loss", loss, on_epoch=True, on_step=True, logger=True)
        self.log("test_acc", acc, on_epoch=True, on_step=True, logger=True)


In [4]:
def find_best_checkpoint(cp_path) -> (str, int):
    '''Returns the best checkpoint file with the number of cumulative epoch'''
    if next(p(cp_path).iterdir(), None) is None:
        return None, 0
    acc_pattern = re.compile(r'(?<=val_loss\=)\d.\d{2}')
    cps = [str(cp) for cp in p(cp_path).glob('*')]
    cps = sorted(cps, key=lambda line: re.search(acc_pattern, line).group())
    best = cps[0]
    resumed_epoch = re.search('\d+', best.split(',', 1)[0]).group()
    with open(f'{p(cp_path) / "Resumed epoch.txt"}', 'w') as f:
        f.write(resumed_epoch)
    return best, int(resumed_epoch)

In [5]:
data_path = "Hand extraction/source_box"
model_cp_path = "Model Checkpoints"
model_cp_filename = "Image_model"
data_module = ImageDataset(data_path)
check_point_frequency = 10
best_cpt, resumed_epoch = find_best_checkpoint(model_cp_path)
print(f'resumed_epoch = {resumed_epoch}')
checkpoint_params = {'filename': '{epoch}, {val_loss:.2f}, {val_acc:.2f}', 'save_top_k': 5,
                     'monitor': "val_acc",
                     'mode': 'max',
                     'every_n_epochs': check_point_frequency, 'save_on_train_epoch_end': True}
model = Resnet()
if best_cpt:
    model = Resnet.load_from_checkpoint(best_cpt)
trainer = pl.Trainer(
    max_epochs=500 - resumed_epoch,
    auto_lr_find=True,
    accelerator='gpu',
    devices=-1,
    auto_select_gpus=True,
    auto_scale_batch_size='binsearch',
    check_val_every_n_epoch=check_point_frequency,
    detect_anomaly=True,
    precision=16,
    callbacks=ModelCheckpoint(dirpath=model_cp_path, **checkpoint_params),
)
trainer.fit(model, data_module)

resumed_epoch = 0


Auto select gpus: [0]
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | acc   | Accuracy         | 0     
1 | loss  | CrossEntropyLoss | 0     
2 | model | ResNet           | 11.7 M
-------------------------------------------
11.7 M    Trainable params
0         Non-trainable params
11.7 M    Total params
23.379    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\dtngh\miniconda3\envs\deep_learning\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:229: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
C:\Users\dtngh\miniconda3\envs\deep_learning\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:229: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
C:\Users\dtngh\miniconda3\envs\deep_learning\lib\site-packages\pytorch_lightning\trainer\trainer.py:1562: PossibleUserWarning: The number of trainin

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

C:\Users\dtngh\miniconda3\envs\deep_learning\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


import webbrowser

# Open a url after done
webbrowser.open('https://youtu.be/5dwxGvmUG90?t=53')

In [7]:
import os

os.system("shutdown /s /t 1")

0

1190